# Problems: NGS
https://rosalind.info/problems/topics/ngs/

## 1. FASTQ format introduction

https://rosalind.info/problems/tfsq/

In [15]:
with open("output_fasta.fasta", "w") as fw:
    for record in SeqIO.parse("rosalind_tfsq.txt", "fastq"):
        fw.write(f">{record.description}\n{str(record.seq)}\n")


## 2. Read Quality Distribution

https://rosalind.info/problems/phre/

In [18]:
from io import StringIO

In [26]:
with open("rosalind_phre.txt", "r") as fi:
    threshold = int(fi.readline().strip())
    fastq_handle = StringIO(fi.read())
    count = 0  # count #reads with  whose avg quality is below threshold
    for record in SeqIO.parse(fastq_handle, "fastq"):
        quality = record.letter_annotations['phred_quality']
        avg_quality = sum(quality) / len(quality)
        if avg_quality < threshold:
            count += 1
print(count)

44


## 3. Read Filtration by Quality

https://rosalind.info/problems/filt/

In [33]:
with open("rosalind_filt.txt", "r") as fi:
    threshold, p = list(map(int, fi.readline().strip().split()))
    fastq_handle = StringIO(fi.read())
    count = 0  # count #filtered reads
    for record in SeqIO.parse(fastq_handle, "fastq"):
        quality = record.letter_annotations['phred_quality']
        passed_quality = [qual for qual in quality if qual >= threshold]
        read_percentage = (len(passed_quality) / len(quality)) * 100
        if read_percentage >= p:
            count += 1

print(count)


67


## 4. Base Quality Distribution

https://rosalind.info/problems/bphr/

In [37]:
with open("rosalind_bphr.txt", "r") as fi:
    threshold = int(fi.readline().strip())
    fastq_handle = StringIO(fi.read())
    qualities = []
    for record in SeqIO.parse(fastq_handle, "fastq"):
        qualities.append(record.letter_annotations['phred_quality'])

count = 0  # count #base with  whose mean quality is below threshold
for per_pos_quals in zip(*qualities):
    quality = sum(per_pos_quals) / len(per_pos_quals)
    if quality < threshold:
        count += 1
print(count)

6


## 5. Base Filtration by Quality

https://rosalind.info/problems/bfil/

In [53]:
from Bio import SeqIO
from io import StringIO

In [52]:
def trim_read(record, quality_threshold):
    """
    Trim the leading and trailing bases from a FASTQ read based on a quality threshold.

    Parameters:
    record (SeqRecord): A SeqRecord object from a FASTQ file.
    quality_threshold (int): The quality score threshold below which bases are trimmed.

    Returns:
    tuple: A tuple containing the trimmed sequence (str) and the trimmed quality string (str).
    """
    seq = str(record.seq)
    quality_scores = record.letter_annotations["phred_quality"]
    
    # Find the first base with quality >= threshold
    start = 0
    while start < len(quality_scores) and quality_scores[start] < quality_threshold:
        start += 1

    # Find the last base with quality >= threshold
    end = len(quality_scores) - 1
    while end >= 0 and quality_scores[end] < quality_threshold:
        end -= 1

    # Trim the sequence and quality scores
    if start <= end:
        trimmed_seq = seq[start:end+1]
        trimmed_qualities = quality_scores[start:end+1]
    else:
        trimmed_seq = ""
        trimmed_qualities = []

    trimmed_qualities = ''.join(chr(score + 33) for score in trimmed_qualities)
    return (trimmed_seq, trimmed_qualities)

with open("rosalind_bfil.txt", "r") as fi:
    q = int(fi.readline().strip())
    fastq_content = fi.read()

# Write the trimmed records to the output file
with open("output.fastq", "w") as fw:
    fastq_handle = StringIO(fastq_content)
    for record in SeqIO.parse(fastq_handle, "fastq"):
        trimmed_seq, trimmed_qualities = trim_read(record, q)
        if trimmed_seq:  # Only write records with remaining sequence
            fw.write(f"@{record.description}\n{trimmed_seq}\n+\n{trimmed_qualities}\n")
